# Predictive Modeling Database Creation Notebook

## Description:

This notebook serves as a guide for creating databases tailored specifically for predictive modeling tasks. The primary objective is to design and build databases that support machine learning models by providing well-structured data and meaningful features.

In this notebook, we'll cover the process of database creation with a focus on enhancing data quality and feature engineering. 

Additionally, we'll explore methods for generating and incorporating features into the database to enhance the predictive power of machine learning models. These features may include engineered attributes, transformed variables, or aggregated statistics derived from the raw data.


**Author:** [Caique Matos]
**Date:** [04/16/24]


In [1]:
import pandas as pd
import numpy as np
import copy as cp
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import math
import spacy


from sklearn.model_selection import train_test_split, StratifiedKFold

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer, PorterStemmer

from sklearn.metrics import jaccard_score
import itertools

import re

import scipy.stats as ss
from scipy.stats import kruskal

from sklearn.preprocessing import OrdinalEncoder
import pickle

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [2]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim import corpora

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [4]:
path = 'C:/Users/caiqu/OneDrive/Competitions/sony/input/dataset/'

In [5]:
df_train = pd.read_excel(path+"train_set.xlsx")

In [6]:
df_test = pd.read_excel(path+"test_set.xlsx")

In [7]:
df_train.columns

Index(['ID', 'SRC_TITLE_ID', 'SRC_TITLE_NM', 'TITLE_TYPE', 'TITLE_YR',
       'RELEASE_DT', 'RUN_TIME', 'PLOT_OUTLINE', 'PLOT_MEDIUM', 'PLOT_SUMMARY',
       'RATING_AVG', 'NO_OF_VOTES', 'BUDGET_AMT', 'SRC_GENRE'],
      dtype='object')

In [8]:
df_test.columns

Index(['ID', 'SRC_TITLE_ID', 'SRC_TITLE_NM', 'TITLE_TYPE', 'TITLE_YR',
       'RELEASE_DT', 'RUN_TIME', 'PLOT_OUTLINE', 'PLOT_MEDIUM', 'PLOT_SUMMARY',
       'RATING_AVG', 'NO_OF_VOTES', 'BUDGET_AMT'],
      dtype='object')

Este notebook é dedicado em montar a base de dados que sera utilizada pelo modelo preditor de generos.
Utilizaremos os conhecimentos descobertos durante o EDA para extrair feature, tratar os dados textuais e demais necessidades de tratamento de dados.


## Feature Engineering

### Handle Missing

In [9]:
# Filling in missing values ​​with the median
df_train['RUN_TIME'].fillna(df_train['RUN_TIME'].median(), inplace=True)
df_test['RUN_TIME'].fillna(df_test['RUN_TIME'].median(), inplace=True)

#We will use the average per hour, but we may choose to remove due to the large number of missing films
df_train['BUDGET_AMT'].fillna(df_train['BUDGET_AMT'].median(), inplace=True)
df_test['BUDGET_AMT'].fillna(df_test['BUDGET_AMT'].median(), inplace=True)


df_train['PLOT_OUTLINE'].fillna('', inplace=True)
df_test['PLOT_OUTLINE'].fillna('', inplace=True)

df_train['PLOT_MEDIUM'].fillna('', inplace=True)
df_test['PLOT_MEDIUM'].fillna('', inplace=True)

df_train['PLOT_SUMMARY'].fillna('', inplace=True)
df_test['PLOT_SUMMARY'].fillna('', inplace=True)

### Temporal Features

In [10]:
# Converting 'RELEASE_DT' to date format in df_train
df_train['RELEASE_DT'] = pd.to_datetime(df_train['RELEASE_DT'])

# Extracting the day and month in df_train
df_train['RELEASE_DAY'] = df_train['RELEASE_DT'].dt.day
df_train['RELEASE_MONTH'] = df_train['RELEASE_DT'].dt.month


df_test['RELEASE_DT'] = pd.to_datetime(df_test['RELEASE_DT'])

df_test['RELEASE_DAY'] = df_test['RELEASE_DT'].dt.day
df_test['RELEASE_MONTH'] = df_test['RELEASE_DT'].dt.month

### Top terms

In [11]:
# Function to calculate the most common terms in a text
def calculate_top_terms(text_column, num_top_terms=20):
    # Replace NaN values with an empty string
    text_column = text_column.fillna('')
    
    # Text preprocessing (removing stopwords, punctuation, etc.) and tokenization
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(text_column.astype(str))  # Converting to string
    
    # Calculate word frequencies
    word_frequencies = X.sum(axis=0)

    # Get the most common terms
    terms = vectorizer.get_feature_names_out()
    word_freq_df = pd.DataFrame({'Term': terms, 'Frequency': word_frequencies.flat})
    word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)
    
    # Return the most common terms
    return word_freq_df['Term'][:num_top_terms].tolist()

# Function to add columns of most common terms to a DataFrame
def add_top_terms_columns(df, text_columns, num_top_terms=20):
    for column in text_columns:
        # Calculate the most common terms for the column
        top_terms = calculate_top_terms(df[column], num_top_terms)
        
        # Add the most common terms as a new column in the DataFrame
        df[f'TOP_TERMS_{column}'] = df[column].apply(lambda text: [term for term in top_terms if isinstance(text, str) and term in text.split()])

In [12]:
# List of text columns to process
text_columns = ['PLOT_SUMMARY', 'PLOT_OUTLINE', 'PLOT_MEDIUM', 'SRC_TITLE_NM']

### Sentiment Analysis

In [13]:
def analyze_sentiment(df, column_name):
    # Initialize the sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    # Function to assign a sentiment label based on the sentiment compound score
    def sentiment_label(score):
        if score >= 0.05:
            return 'Positive'
        elif score <= -0.05:
            return 'Negative'
        else:
            return 'Neutral'

    # Calculate sentiment for each entry in the specified column
    sentiments = []
    for text in df[column_name]:
        # Calculate sentiment score
        sentiment_score = sia.polarity_scores(str(text))['compound']
        # Assign a sentiment label
        label = sentiment_label(sentiment_score)
        sentiments.append(label)

    # Add sentiment labels to the DataFrame
    df[column_name + '_SENTIMENT'] = sentiments

In [14]:
analyze_sentiment(df_train, 'PLOT_OUTLINE')
analyze_sentiment(df_test, 'PLOT_OUTLINE')

analyze_sentiment(df_train, 'PLOT_MEDIUM')
analyze_sentiment(df_test, 'PLOT_MEDIUM')

analyze_sentiment(df_train, 'PLOT_SUMMARY')
analyze_sentiment(df_test, 'PLOT_SUMMARY')

analyze_sentiment(df_train, 'SRC_TITLE_NM')
analyze_sentiment(df_test, 'SRC_TITLE_NM')

In [15]:
def encode_categorical_columns(df, categorical_columns, encoder_filename):
    
    encoder = OrdinalEncoder()
    encoder.fit(df[categorical_columns])
    encoded_data = encoder.transform(df[categorical_columns])
    
    
    # Save the encoder model
#     with open(encoder_filename, 'wb') as file:
#         pickle.dump(encoder, file)
    
    for i, column in enumerate(categorical_columns):
        new_column_name = f"{column}_ENCODED"
        df[new_column_name] = encoded_data[:, i]+1
    
    return df

In [16]:
#mapping categorical to number

categorical_columns = ['PLOT_SUMMARY_SENTIMENT','SRC_TITLE_NM_SENTIMENT','PLOT_OUTLINE_SENTIMENT','PLOT_MEDIUM_SENTIMENT']

df_train = encode_categorical_columns(df_train, categorical_columns, 'encoder_fulltrain.pkl')
df_test = encode_categorical_columns(df_test, categorical_columns, 'encoder_fulltest.pkl')

### Entities and Dominant Topics

In [17]:
# Function to get the dominant topic of a text
def get_dominant_topic(lda_model, text):
    # Converting the text into a bag-of-words format
    bow = dictionary.doc2bow(text.split())

    # Getting the topic distribution for the document
    topic_distribution = lda_model.get_document_topics(bow)

    # Returning the topic with the highest score
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]

    return dominant_topic

In [18]:
# Loop over the text columns
for column in text_columns:
    # Convert the values in the column to string
    texts = df_train[column].astype(str).tolist()
    tokens = [text.split() for text in texts]
    dictionary = Dictionary(tokens)

    # Filter extreme tokens in the dictionary (optional)
    dictionary.filter_extremes(no_below=5, no_above=0.5)

    # Create a corpus from the texts
    corpus = [dictionary.doc2bow(tokens) for tokens in tokens]

    # Train the LDA model
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=10)

    # Adding a new column 'DOMINANT_TOPIC' to the DataFrame
    df_train[column + '_DOMINANT_TOPIC'] = [get_dominant_topic(lda_model, text) for text in texts]


In [19]:
# Loop over the text columns
for column in text_columns:
    # Convert the values in the column to string
    texts = df_test[column].astype(str).tolist()
    tokens = [text.split() for text in texts]
    dictionary = Dictionary(tokens)

    # Filter extreme tokens in the dictionary (optional)
    dictionary.filter_extremes(no_below=5, no_above=0.5)

    # Create a corpus from the texts
    corpus = [dictionary.doc2bow(tokens) for tokens in tokens]

    # Train the LDA model
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=10)

    # Adding a new column 'DOMINANT_TOPIC' to the DataFrame
    df_test[column + '_DOMINANT_TOPIC'] = [get_dominant_topic(lda_model, text) for text in texts]


In [20]:
scaler = MinMaxScaler()

df_train['RATING_AVG'] = scaler.fit_transform(df_train[['RATING_AVG']])
df_test['RATING_AVG'] = scaler.fit_transform(df_test[['RATING_AVG']])

### Vectorizing Text Columns

This step should be at The models notebooks due to specificity of each models embedding.


### Umbalance Class in Genres

Class imbalance in movie genres occurs when some genres have significantly fewer instances compared to others. This can lead to biased models that perform poorly in predicting underrepresented genres.

To address this issue, under sampling reduces the number of instances in the majority class, while oversampling increases the instances in the minority class. Both techniques aim to balance the class distribution and improve model performance.

### Under Sampling

In [21]:
# Obter os 12 gêneros com mais amostras
top_genres = df_train['SRC_GENRE'].value_counts().nlargest(7).index

# Filtrar o DataFrame original para incluir apenas os filmes desses gêneros
df_under = df_train[df_train['SRC_GENRE'].isin(top_genres)].copy()

In [22]:
categorical_columns = ['SRC_GENRE']
df_under = encode_categorical_columns(df_under, categorical_columns, 'encoder_2.pkl')

In [23]:
df_under['SRC_GENRE'].value_counts()

Drama          3131
Documentary    2829
Comedy         2277
Action         1120
Horror          696
Crime           440
Adventure       284
Name: SRC_GENRE, dtype: int64

In [24]:
df_validation, df_under = train_test_split(df_under, test_size=0.95, random_state=12)
df_validation.to_csv(path+ '../df_validation_under.csv', index=False)

In [25]:
df_test.to_csv(path+ '../df_test.csv', index=False)
df_under.to_csv(path+ '../df_train_under.csv', index=False)

In [26]:
df_under.head()

,ID,SRC_TITLE_ID,SRC_TITLE_NM,TITLE_TYPE,TITLE_YR,RELEASE_DT,RUN_TIME,PLOT_OUTLINE,PLOT_MEDIUM,PLOT_SUMMARY,RATING_AVG,NO_OF_VOTES,BUDGET_AMT,SRC_GENRE,RELEASE_DAY,RELEASE_MONTH,PLOT_OUTLINE_SENTIMENT,PLOT_MEDIUM_SENTIMENT,PLOT_SUMMARY_SENTIMENT,SRC_TITLE_NM_SENTIMENT,PLOT_SUMMARY_SENTIMENT_ENCODED,SRC_TITLE_NM_SENTIMENT_ENCODED,PLOT_OUTLINE_SENTIMENT_ENCODED,PLOT_MEDIUM_SENTIMENT_ENCODED,PLOT_SUMMARY_DOMINANT_TOPIC,PLOT_OUTLINE_DOMINANT_TOPIC,PLOT_MEDIUM_DOMINANT_TOPIC,SRC_TITLE_NM_DOMINANT_TOPIC,SRC_GENRE_ENCODED
11004,11005,tt9618458,Night Blooms,movie,2022,2021-09-18,98.00,Jessica Clement stars with Nick Stahl in a 90'...,17 year old Carly initiates a romantic relatio...,Jessica Clement stars with Nick Stahl in a 90'...,0.52,157,300000.00,Drama,18.00,9.00,Positive,Positive,Positive,Neutral,3.00,2.00,3.00,3.00,6,8,9,6,6.00
10716,10717,tt2411112,Longwave,movie,2013,2013-07-19,85.00,,"April 1974. The Federal Councellor, annoyed by...","April 1974. The Federal Councellor, annoyed by...",0.64,623,300000.00,Comedy,19.00,7.00,Neutral,Negative,Negative,Neutral,1.00,2.00,2.00,1.00,7,0,7,0,3.00
7217,7218,tt7197162,Old Man Jackson,movie,2023,2023-06-01,92.00,"Mr. Jackson, an award winning automotive mecha...",,"Mr. Jackson, an award winning automotive mecha...",0.88,70,2500000.00,Comedy,1.00,6.00,Positive,Neutral,Positive,Neutral,3.00,2.00,3.00,2.00,7,7,0,1,3.00
3367,3368,tt1283884,Blodigt jÃ¤vla helvete,movie,2008,2008-02-15,108.00,A rookie cop arrives to the city of Gotland to...,,A rookie cop arrives to the city of Gotland to...,0.67,41,300000.00,Action,15.00,2.00,Negative,Neutral,Negative,Neutral,1.00,2.00,1.00,2.00,2,2,0,0,1.00
8557,8558,tt1384925,Ardor,movie,2014,2014-05-12,101.00,A mysterious man emerges from the Argentinean ...,Bordering the vast Amazonian rainforest along ...,A mysterious man emerges from the Argentinean ...,0.52,907,300000.00,Drama,12.00,5.00,Negative,Positive,Negative,Neutral,1.00,2.00,1.00,3.00,2,2,9,0,6.00


### Over Sampling

In [27]:
# Class Balancing

from sklearn.utils import resample

# Determine the minimum number of observations for each genre
min_obs_per_genre = 1000

# Count the number of observations for each genre
genre_counts = df_train['SRC_GENRE'].value_counts()

# List the genres with few observations
genres_to_oversample = genre_counts[genre_counts < min_obs_per_genre].index.tolist()

# Create artificial rows for each genre with few observations
for genre in genres_to_oversample:
    # Filter the observations for the specific genre
    df_genre = df_train[df_train['SRC_GENRE'] == genre]
    
    # Determine how many artificial observations are needed to equal the minimum number
    num_samples_needed = min_obs_per_genre - len(df_genre)
    
    # Perform oversampling to create artificial rows
    df_genre_oversampled = resample(df_genre, 
                                     replace=True,
                                     n_samples=num_samples_needed,
                                     random_state=42)
    
    # Add the artificial rows to the original DataFrame
    df_train = pd.concat([df_train, df_genre_oversampled])

In [28]:
categorical_columns = ['SRC_GENRE']
df_train = encode_categorical_columns(df_train, categorical_columns,'encoder_3.pkl')

In [29]:
df_validation, df_train = train_test_split(df_train, test_size=0.95, random_state=42)

df_train.to_csv(path+ '../df_train_over.csv', index=False)
df_validation.to_csv(path+ '../df_validation_over.csv', index=False)